In [8]:
from load_data import *


df = load_data()
df = df.drop("name", axis=1)
df = df.drop("notes", axis=1)

event_locations = df['event_location'].unique()
len(event_locations)

df.sort_values(by='age', ascending=False)[0:4]

2023-11-09 15:29:46.456 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-11-09 15:29:46.495 
  command:

    streamlit run /home/silas/Documents/school/minor/student_alcohol_consumption/.venv/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2023-11-09 15:29:46.496 No runtime found, using MemoryCacheStorageManager


,date_of_event,age,citizenship,event_location,event_location_district,event_location_region,date_of_death,gender,took_part_in_the_hostilities,place_of_residence,place_of_residence_district,type_of_injury,ammunition,killed_by
2597,2014-07-29,112.0,Palestinian,Khuza'ah,Khan Yunis,Gaza Strip,2014-08-02,F,No,Khuza'a,Khan Yunis,gunfire,NaN,Israeli security forces
5616,2008-12-27,101.0,Palestinian,Gaza City,Gaza,Gaza Strip,2008-12-27,M,No,al-Bureij R.C.,Deir al-Balah,shelling,NaN,Israeli security forces
9475,2002-12-03,94.0,Palestinian,al-Birah,Ramallah and al-Bira,West Bank,2002-12-03,F,No,Ramallah,Ramallah and al-Bira,gunfire,NaN,Israeli security forces
2196,2014-08-01,94.0,Palestinian,Khuza'ah,Khan Yunis,Gaza Strip,2014-08-01,M,No,Khuza'a,Khan Yunis,gunfire,NaN,Israeli security forces


In [6]:
import string
from requests.structures import CaseInsensitiveDict
import urllib.parse
import requests

API_KEY = "9f9e43cd863749edb81f777af21853b0"


def find_longLat(location: string):
    url = f"https://api.geoapify.com/v1/geocode/search?text={urllib.parse.quote(location)}&apiKey={API_KEY}&filter=countrycode:il,ps"

    headers = CaseInsensitiveDict()
    headers["Accept"] = "application/json"

    resp = requests.get(url, headers=headers)

    return resp.json()

In [10]:
responses = []

for loc in event_locations:
    resp = find_longLat(loc)
    resp['originalQuery'] = loc
    responses.append(resp)

responses

[{'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': [35.057856, 32.320626]},
    'properties': {'country_code': 'il',
     'country': 'Israel',
     'county': 'Tulkarm',
     'datasource': {'sourcename': 'whosonfirst',
      'url': 'https://www.whosonfirst.org/docs/licenses/'},
     'state': 'West Bank',
     'city': 'Nur Shams',
     'lon': 35.057856,
     'lat': 32.320626,
     'formatted': 'Nur Shams, Tulkarm, Israel',
     'address_line1': 'Nur Shams',
     'address_line2': 'Tulkarm, Israel',
     'timezone': {'name': 'Asia/Hebron',
      'offset_STD': '+02:00',
      'offset_STD_seconds': 7200,
      'offset_DST': '+03:00',
      'offset_DST_seconds': 10800,
      'abbreviation_STD': 'EET',
      'abbreviation_DST': 'EEST'},
     'plus_code': '8G4Q83C5+74',
     'result_type': 'city',
     'rank': {'popularity': 3.355007640799266,
      'confidence': 0.81,
      'confidence_city_level': 0.81,
      'match_type': 'full_

In [11]:
objs = responses

In [14]:
import json

with open("locations.json", "w") as f:
    json.dump(responses, f)

In [15]:
import json

objs = []
with open("locations.json", "r") as f:
    objs = json.load(f)

In [16]:
missingfeatures = [x for x in objs if len(x['features']) < 1]
completefeatures = [x['features'][0]['properties']
                    for x in objs if len(x['features']) >= 1]

In [17]:
completefeatures

[{'country_code': 'il',
  'country': 'Israel',
  'county': 'Tulkarm',
  'datasource': {'sourcename': 'whosonfirst',
   'url': 'https://www.whosonfirst.org/docs/licenses/'},
  'state': 'West Bank',
  'city': 'Nur Shams',
  'lon': 35.057856,
  'lat': 32.320626,
  'formatted': 'Nur Shams, Tulkarm, Israel',
  'address_line1': 'Nur Shams',
  'address_line2': 'Tulkarm, Israel',
  'timezone': {'name': 'Asia/Hebron',
   'offset_STD': '+02:00',
   'offset_STD_seconds': 7200,
   'offset_DST': '+03:00',
   'offset_DST_seconds': 10800,
   'abbreviation_STD': 'EET',
   'abbreviation_DST': 'EEST'},
  'plus_code': '8G4Q83C5+74',
  'result_type': 'city',
  'rank': {'popularity': 3.355007640799266,
   'confidence': 0.81,
   'confidence_city_level': 0.81,
   'match_type': 'full_match'},
  'place_id': '514ef04dd36787414059a71fd4450a294040c00208e2031e77686f736f6e66697273743a6c6f63616c6974793a313031393231313839'},
 {'country_code': 'il',
  'country': 'Israel',
  'county': 'Sharon',
  'datasource': {'source

In [18]:
len(missingfeatures)

27

In [19]:
import pandas as pd

df = pd.DataFrame(completefeatures)

df.head()

,country_code,country,county,datasource,state,city,lon,lat,formatted,address_line1,...,street,suburb,plus_code_short,distance,district,old_name,isolated_dwelling,housenumber,retail,industrial
0,il,Israel,Tulkarm,"{'sourcename': 'whosonfirst', 'url': 'https://...",West Bank,Nur Shams,35.057856,32.320626,"Nur Shams, Tulkarm, Israel",Nur Shams,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,il,Israel,Sharon,"{'sourcename': 'geonames', 'license': 'CC BY 4...",Central,Kfar Yona,34.935070,32.316690,"Kfar Yona, Sharon, Israel",Kfar Yona,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,il,Israel,Jericho,"{'sourcename': 'whosonfirst', 'url': 'https://...",West Bank,Aqabat Jaber,35.441459,31.838383,"Aqabat Jaber, Jericho, Israel",Aqabat Jaber,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ps,Palestinian Territory,Area A,"{'sourcename': 'openstreetmap', 'attribution':...",West Bank,Jenin,35.297566,32.461884,"180 Jenin, Palestinian Territory",Jenin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ps,Palestine,Gaza,"{'sourcename': 'whosonfirst', 'url': 'https://...",Gaza Strip,Gaza City,34.453956,31.508115,"Gaza City, Gaza, Palestine",Gaza City,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map()
cluster = MarkerCluster().add_to(m)

for i, row in df.iterrows():
    folium.Marker(location=[row['lat'], row['lon']]).add_to(cluster)

m

In [2]:
import pandas as pd

df = pd.read_csv('./fatalities_isr_pse_conflict_2000_to_2023.csv')
df

,name,date_of_event,age,citizenship,event_location,event_location_district,event_location_region,date_of_death,gender,took_part_in_the_hostilities,place_of_residence,place_of_residence_district,type_of_injury,ammunition,killed_by,notes
0,'Abd a-Rahman Suleiman Muhammad Abu Daghash,2023-09-24,32.0,Palestinian,Nur Shams R.C.,Tulkarm,West Bank,2023-09-24,M,NaN,Nur Shams R.C.,Tulkarm,gunfire,live ammunition,Israeli security forces,Fatally shot by Israeli forces while standing ...
1,Usayed Farhan Muhammad 'Ali Abu 'Ali,2023-09-24,21.0,Palestinian,Nur Shams R.C.,Tulkarm,West Bank,2023-09-24,M,NaN,Nur Shams R.C.,Tulkarm,gunfire,live ammunition,Israeli security forces,Fatally shot by Israeli forces while trying to...
2,'Abdallah 'Imad Sa'ed Abu Hassan,2023-09-22,16.0,Palestinian,Kfar Dan,Jenin,West Bank,2023-09-22,M,NaN,al-Yamun,Jenin,gunfire,live ammunition,Israeli security forces,Fatally shot by soldiers while firing at them ...
3,Durgham Muhammad Yihya al-Akhras,2023-09-20,19.0,Palestinian,'Aqbat Jaber R.C.,Jericho,West Bank,2023-09-20,M,NaN,'Aqbat Jaber R.C.,Jericho,gunfire,live ammunition,Israeli security forces,Shot in the head by Israeli forces while throw...
4,Raafat 'Omar Ahmad Khamaisah,2023-09-19,15.0,Palestinian,Jenin R.C.,Jenin,West Bank,2023-09-19,M,NaN,Jenin,Jenin,gunfire,live ammunition,Israeli security forces,Wounded by soldiers’ gunfire after running awa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11119,Binyamin Herling,2000-10-19,64.0,Israeli,Nablus,Nablus,West Bank,2000-10-19,M,Israelis,Kedumim,Tulkarm,gunfire,live ammunition,Palestinian civilians,Killed while hiking on Mt. Eival.
11120,Farid Musa 'Issa a-Nesasreh,2000-10-17,28.0,Palestinian,Beit Furik,Nablus,West Bank,2000-10-17,M,Unknown,Beit Furik,Nablus,gunfire,NaN,Israeli civilians,Killed by a settler from Itamar while harvesti...
11121,Hillel Lieberman,2000-10-07,36.0,Israeli,Nablus,Nablus,West Bank,2000-10-07,M,Israelis,Elon Moreh,Nablus,gunfire,live ammunition,Palestinian civilians,His body was found a day after he disappeared.
11122,Fahed Mustafa 'Odeh Baker,2000-10-07,21.0,Palestinian,Bidya,Salfit,West Bank,2000-10-07,M,No,Bidya,Salfit,gunfire,NaN,Israeli civilians,Killed by settlers who rioted in Biddya village.
